In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### **Data Exploration**

In [ ]:
%run config/seaborn_config.ipynb

##### Univariate Distributions

In [3]:
# Main function
def explore_univariate_distribution(
    data: pd.DataFrame,
    columns: list,
    dist_type: str,
    relative_frequency: bool = True,
    n_bins: int = 20,
):
    """
    Main function to explore univariate distributions (discrete or continuous).

    Args:
    - data: A pandas DataFrame.
    - columns: A list of column names to explore.
    - dist_type: Distribution type: "discrete" or "continuous".
    - relative_frequency: If True, use relative frequencies for discrete variables.
    - n_bins: Number of bins for continuous distributions.
    """
    if dist_type == "discrete":
        for col in columns:
            # Explore discrete variable
            explore_discrete(data, col, relative_frequency)

    elif dist_type == "continuous":
        for col in columns:
            # Explore continuous variable
            explore_continuous(data, col, n_bins)
            
    else:
        raise ValueError("dist_type must be either 'discrete' or 'continuous'.")

In [ ]:
def explore_discrete(data: pd.DataFrame, col: str, relative_frequency: bool):
    """
    Function to explore the distribution of a discrete variable.

    Args:
    - data: A pandas DataFrame.
    - col: The column to explore.
    - relative_frequency: If True, use relative frequencies, otherwise use absolute.
    """
    # Configure the barplot
    stat, fmt, ylim = configure_barplot(data, col, relative_frequency)

    # Plot the bar plot
    plot_barplot(data, col, stat, ylim, fmt)

    # Add cardinality information to the barplot
    add_cardinality_info(data, col)

    # Show the barplot
    plt.show()

In [ ]:
# Sub-function to configure barplot
def configure_barplot(data: pd.DataFrame, col: str, relative_frequency: bool):
    """
    Configure plot settings for discrete distributions.

    Args:
    - data: A pandas DataFrame.
    - col: The column to explore.
    - relative_frequency: If True, use relative frequencies, otherwise absolute.

    Returns:
    - stat: Statistical option for seaborn plot.
    - fmt: Format for bar labels.
    - ylim: Upper limit for y-axis.
    """
    # Configure stat, fmt, ylim
    if relative_frequency:
        stat = "probability"
        fmt = "%.2f"
        ylim = 1
        
    else:
        stat = "count"
        fmt = "%.0f"
        ylim = data[col].value_counts().max() * 1.2

    return stat, fmt, ylim

In [ ]:
# Sub-function to plot barplot
def plot_barplot(data: pd.DataFrame, col: str, stat: str, ylim: float, fmt: str):
    """
    Function to create a bar plot for the discrete variable.

    Args:
    - data: A pandas DataFrame.
    - col: The column to explore.
    - stat: Statistic to display ('count' or 'probability').
    - ylim: Y-axis limit.
    - fmt: Format for bar labels.
    """
    # Create bar plot
    ax = sns.histplot(
        data=data,
        x=col,
        stat=stat,
        discrete=True,
        shrink=0.95,
        alpha=0.50,
        color=custom_colors["blue"],
    )

    # Add value labels
    if hasattr(ax, "containers"):
        for bar in ax.containers:
            ax.bar_label(bar, fmt=fmt, fontsize=10)

    # Add plot aesthetics
    plt.title(f"Distribution of **{col}**")
    plt.xlabel(col)
    plt.xticks(data[col].unique().dropna().sort_values(ascending=True))
    plt.ylabel("Frequency")
    plt.ylim(0, ylim)

In [ ]:
# Sub-function to add cardinality information to barplot
def add_cardinality_info(data: pd.DataFrame, col: str):
    """Adds cardinality information to the barplot."""
    # Calculate number of unique values
    cardinality = data[col].nunique()
    
    # Add number of unique values to the plot
    plt.text(
        x=1.02,
        y=0.90,
        s=f"Unique values: {cardinality}",
        transform=plt.gca().transAxes,
        color="#000000",
    )

    # Add indication for moderate or high cardinality to the plot
    if cardinality >= 10 and cardinality < 50:
        plt.text(
            x=1.02,
            y=0.80,
            s="Moderate Cardinality",
            transform=plt.gca().transAxes,
            color=custom_colors["orange"],
        )
    elif cardinality >= 50:
        plt.text(
            x=1.02,
            y=0.80,
            s="High Cardinality",
            transform=plt.gca().transAxes,
            color=custom_colors["red"],
        )

In [ ]:
# Refactored function to explore continuous variables
def explore_continuous(data: pd.DataFrame, col: str, n_bins: int):
    """
    Function to explore the distribution of a continuous variable.

    Args:
    - data: A pandas DataFrame.
    - col: The column to explore.
    - n_bins: Number of bins to use for the histogram.
    """
    # Calculate continuous statistics
    stats = calculate_continuous_stats(data, col)

    # Plot the histplot
    plot_histplot(data, col, n_bins)

    # Add the continuous stats information
    add_continuous_stats_info(stats)

    # Add legend
    plt.legend()

    # Show the plot
    plt.show()


In [ ]:
# Sub-function to calculate continuous statistics
def calculate_continuous_stats(data: pd.DataFrame, col: str):
    """
    Calculate basic statistics for a continuous variable.

    Args:
    - data: A pandas DataFrame.
    - col: The column to calculate statistics for.

    Returns:
    - A dictionary of calculated statistics.
    """
    # Calculate statistics
    stats = {
        "min": data[col].min(),
        "max": data[col].max(),
        "mean": data[col].mean(),
        "median": data[col].median(),
        "mode": data[col].mode()[0],
        "var": data[col].var(),
        "std": data[col].std(),
        "skew": data[col].skew(),
        "kurt": data[col].kurt(),
    }
    return stats

In [ ]:
# Sub-function to plot histplot
def plot_histplot(data: pd.DataFrame, col: str, n_bins: int):
    """
    Function to create a histogram plot for the continuous variable.

    Args:
    - data: A pandas DataFrame.
    - col: The column to explore.
    - n_bins: Number of bins to use for the histogram.
    """
    # Create histogram
    ax = sns.histplot(
        data=data,
        x=col,
        bins=n_bins,
        kde=True,
        stat="probability",
        alpha=0.50,
        color=custom_colors["blue"],
    )
    
    # Add plot aesthetics
    plt.title(f"Distribution of **{col}**")
    plt.xlabel(col)
    plt.ylabel("Frequency")

In [ ]:
# Sub-function to add statistical annotations to plots
def add_continuous_stats_info(stats: dict):
    """
    Add annotations (mean, median, etc.) to the plot.

    Args:
    - stats: A dictionary of calculated statistics.
    """
    # Add statistical annotations to the plot
    plt.axvline(
        stats["mean"],
        color="#000000",
        linestyle="-",
        alpha=1.0,
        label=f"Mean: {stats['mean']:.1f}",
    )
    plt.axvline(
        stats["median"],
        color="#000000",
        linestyle="--",
        alpha=1.00,
        label=f"Median: {stats['median']:.1f}",
    )
    plt.axvline(
        stats["mode"],
        color="#000000",
        linestyle=":",
        alpha=1.00,
        label=f"Mode: {stats['mode']:.1f}",
    )

    plt.text(
        x=1.02,
        y=0.80,
        s=f"Min: {stats['min']:.1f}",
        transform=plt.gca().transAxes,
        color="#000000",
    )
    plt.text(
        x=1.02,
        y=0.75,
        s=f"Max: {stats['max']:.1f}",
        transform=plt.gca().transAxes,
        color="#000000",
    )
    plt.text(
        x=1.02,
        y=0.50,
        s=f"Variance: {stats['var']:.1f}",
        transform=plt.gca().transAxes,
        color="#000000",
    )
    plt.text(
        x=1.02,
        y=0.45,
        s=f"Std Dev: {stats['std']:.1f}",
        transform=plt.gca().transAxes,
        color="#000000",
    )
    plt.text(
        x=1.02,
        y=0.20,
        s=f"Skewness: {stats['skew']:.1f}",
        transform=plt.gca().transAxes,
        color="#000000",
    )
    plt.text(
        x=1.02,
        y=0.15,
        s=f"Kurtosis: {stats['kurt']:.1f}",
        transform=plt.gca().transAxes,
        color="#000000",
    )

##### Bivariate Relationships

In [ ]:
# Main function
def explore_bivariate_relationships(
    data: pd.DataFrame,
    categorical_columns: list,
    numerical_columns: list,
    y_column: str,
    y_type: str,
):
    """
    Main function to explore bivariate relationships in a DataFrame based on the target variable's type.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data to analyze.
    categorical_columns (list): List of categorical column names.
    numerical_columns (list): List of numerical column names.
    y_column (str): The name of the target variable column.
    y_type (str): The type of the target variable ('continuous' or 'discrete').
    """
    
    if y_type == "continuous":
        print("Continuous features:")
        # Explore relationships between continuous features and a continuous target variable
        explore_continuous_continuous(data, numerical_columns, y_column)
        
        print("Discrete features:")
        # Explore relationship between discrete features and a continuous target variable
        explore_continuous_discrete(data, categorical_columns, y_column)

    elif y_type == "discrete":
        print("Continuous features:")
        # Explore relationship between continuous features and a discrete target variable
        explore_discrete_continuous(data, numerical_columns, y_column)
        
        print("Discrete features:")
        # Explore relationship between discrete features and a discrete target variable
        explore_discrete_discrete(data, categorical_columns, y_column)

In [ ]:
# Sub-function for relationships between continuous features and a continuous target variable
def explore_continuous_continuous(data, numerical_columns, y_column):
    """
    Explore relationships between continuous features and a continuous target variable.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data to analyze.
    numerical_columns (list): List of numerical column names.
    y_column (str): The name of the target variable column.
    """
    for col in numerical_columns:
        if col == y_column:
            continue
        
        # Calculate correlation coefficients
        pearson_corr, spearman_corr = calculate_correlation_coefficients(
            data, col, y_column
        )
        # Plot the scatterplot
        plot_scatterplot(data, col, y_column, pearson_corr, spearman_corr)

In [ ]:
# Sub-function to calculate correlation coefficients
def calculate_correlation_coefficients(data, col, y_column):
    """
    Calculate Pearson and Spearman correlations.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data.
    col (str): The name of the numerical feature column.
    y_column (str): The name of the target variable column.

    Returns:
    tuple: Pearson and Spearman correlation coefficients.
    """
    # Calculate Pearson correlation
    pearson_corr = data[col].corr(data[y_column], method="pearson")

    # Calculate Spearman correlation
    spearman_corr = data[col].corr(data[y_column], method="spearman")
    
    return pearson_corr, spearman_corr

In [ ]:
# Sub-function to plot a scatterplot
def plot_scatterplot(data, col, y_column, pearson_corr, spearman_corr):
    """
    Plot scatterplot for a continuous feature against the target variable.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data.
    col (str): The name of the numerical feature column.
    y_column (str): The name of the target variable column.
    pearson_corr (float): Pearson correlation coefficient.
    spearman_corr (float): Spearman correlation coefficient.
    """
    # Create scatterplot
    sns.regplot(
        x=col,
        y=y_column,
        data=data,
        ci=None,
        marker="o",
        scatter_kws={"s": 50, "alpha": 0.2},
        line_kws={"color": custom_colors["red"], "linewidth": 2, "linestyle": "--"},
    )

    # Add plot aesthetics
    plt.title(f"Scatterplot of **{col}** / **{y_column}**")
    plt.xlabel(col)
    plt.ylabel(y_column)

    # Add annotations
    plt.text(
        x=1.02,
        y=0.90,
        s=f"Pearson: {pearson_corr:.2f}",
        transform=plt.gca().transAxes,
        color="#000000",
    )
    plt.text(
        x=1.02,
        y=0.80,
        s=f"Spearman: {spearman_corr:.2f}",
        transform=plt.gca().transAxes,
        color="#000000",
    )

    # Show plot
    plt.show()

In [ ]:
# Sub-function for relationships between discrete features and a continuous target variable
def explore_continuous_discrete(data, categorical_columns, y_column):
    """
    Explore relationships between discrete features and a continuous target variable.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data to analyze.
    categorical_columns (list): List of categorical column names.
    y_column (str): The name of the target variable column.
    """
    # Calculate median of target variable
    y_median = data[y_column].median()

    for col in categorical_columns:
        # Plot the grouped boxplot
        plot_grouped_boxplot(data, col, y_column, y_median)

In [ ]:
# Sub-function to plot a boxplot
def plot_grouped_boxplot(data, col, y_column, y_median):
    """
    Plot a boxplot for a discrete feature against a continuous target variable.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data.
    col (str): The name of the categorical feature column.
    y_column (str): The name of the target variable column.
    y_median (float): The median of the target variable.
    """
    # Create boxplot
    sns.boxplot(x=col, y=y_column, data=data, width=0.5)

    # Add plot aesthetics
    plt.title(f"Grouped Boxplot of **{col}** / **{y_column}**")
    plt.xlabel(col)
    plt.ylabel(y_column)
    plt.axhline(y=y_median, color=custom_colors["red"])

    # Show plot
    plt.show()

In [ ]:
# Sub-function for relationships between continuous features and a discrete target variable
def explore_discrete_continuous(data, numerical_columns, y_column):
    """
    Explore relationships between continuous features and a discrete target variable.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data to analyze.
    numerical_columns (list): List of numerical column names.
    y_column (str): The name of the target variable column.
    """
    for col in numerical_columns:
        # Plot the grouped histplot
        plot_grouped_histplot(data, col, y_column)

In [ ]:
# Sub-function to plot a stacked histogram
def plot_grouped_histplot(data, col, y_column):
    """
    Plot a histogram for a continuous feature against a discrete target variable.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data.
    col (str): The name of the numerical feature column.
    y_column (str): The name of the target variable column.
    """
    # Select colors from color palette based on number of categories
    n_categories = data[y_column].nunique()
    selected_colors = custom_qualitative_palette[:n_categories]
    
    # Create grouped histplot
    sns.histplot(
        data=data,
        x=col,
        hue=y_column,
        kde=True,
        stat="probability",
        bins=10,
        common_norm=False,
        alpha=0.50,
        palette=selected_colors,
    )

    # Add plot aesthetics
    plt.title(f"Grouped Histplot of **{col}** / **{y_column}**")
    plt.xlabel(col)
    plt.ylabel(f"Probability of {y_column}")

    # Show plot
    plt.show()

In [ ]:
# Sub-function for relationships between discrete features and a discrete target variable
def explore_discrete_discrete(data, categorical_columns, y_column):
    """
    Explore relationships between discrete features and a discrete target variable.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data to analyze.
    categorical_columns (list): List of categorical column names.
    y_column (str): The name of the target variable column.
    """
    for col in categorical_columns:
        if col == y_column:
            continue
        
        # Plot the stacked barplot
        plot_stacked_barplot(data, col, y_column)

In [ ]:
# Sub-function to plot a stacked barplot
def plot_stacked_barplot(data, col, y_column):
    """
    Plot a stacked bar chart for a discrete feature against a discrete target variable.

    Parameters:
    data (pd.DataFrame): The DataFrame containing the data.
    col (str): The name of the categorical feature column.
    y_column (str): The name of the target variable column.
    """
    # Create a contingency table
    cross_tab = pd.crosstab(data[col], data[y_column], normalize="index")

    # Select colors from color palette based on number of categories
    n_categories = data[y_column].nunique()
    selected_colors = custom_qualitative_palette[:n_categories]

    # Create stacked bar plot
    ax = cross_tab.plot(
        kind="bar", stacked=True, alpha=0.50, color=selected_colors
    )

    # Add value labels
    if hasattr(ax, "containers"):
        for bar in ax.containers:
            ax.bar_label(bar, label_type="center", fmt="%.2f", fontsize=10)

    # Add plot aesthetics
    plt.title(f"Stacked Barplot of **{col}** / **{y_column}**")
    plt.xlabel(col)
    plt.ylabel(f"Probability of {y_column}")
    plt.legend(title=y_column, bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.xticks(rotation=0)

    # Show plot
    plt.show()